## 定义
- 继承关系：继承自 ContextWarpper 类

![image](application_page1.png)

---

## 特点
### 实例创建方式：单例模式
- 每个 Android App 运行时，会首先自动创建 Application 类并实例化 Application 对象，且只有一个
- 也可通过继承 Application 类自定义 Application 类和实例

### 实例形式：全局实例
- 即不同的组件（如 Activity、Service）都可获得 Application 对象且都是同一个对象

### 生命周期：等于 Android App 的生命周期
- Application 对象的生命周期是整个程序中最长的，即等于 Android App的生命周期

---

## 方法介绍

![image](application_page2.png)

### onCreate()
- 调用时刻：Application 实例创建时调用
  - 默认为空实现
- 作用
  - 初始化 应用程序级别 的资源，如全局对象、环境配置变量、图片资源初始化、推送服务的注册等
  - 注：请不要执行耗时操作，否则会拖慢应用程序启动速度
- 数据共享、数据缓存
  - 设置全局共享数据，如全局共享变量、方法等
  - 注：这些共享数据只在应用程序的生命周期内有效，当该应用程序被杀死，这些数据也会被清空，所以只能存储一些具备 临时性的共享数据

```
// 复写方法需要在Application子类里实现
private static final String VALUE = "Carson";
    // 初始化全局变量
    @Override
    public void onCreate() {
        super.onCreate();
        
        VALUE = "Jack";
    }
}
```

### registerComponentCallbacks() & unregisterComponentCallbacks()
- 作用
  - 注册和注销 ComponentCallbacks2 回调接口
  - 本质上是复写 ComponentCallbacks2 回调接口里的方法从而实现更多的操作

```
registerComponentCallbacks(new ComponentCallbacks2() {
    // 接口里方法下面会继续介绍
    @Override
    public void onTrimMemory(int level) {

    }

    @Override
    public void onLowMemory() {

    }

    @Override
    public void onConfigurationChanged(Configuration newConfig) {

    }
});
```

### onTrimMemory()
- 作用：通知 应用程序 当前内存使用情况（以内存级别进行识别）
  - Android 4.0 后提供的一个 API
![image](application_page3.png)

- 应用场景：根据当前内存使用情况进行自身的内存资源的不同程度释放，以避免被系统直接杀掉 & 优化应用程序的性能体验
  - 系统在内存不足时会按照 LRU Cache 中从低到高杀死进程；优先杀死占用内存较高的应用
  - 若应用占用内存较小 = 被杀死几率降低，从而快速启动（即热启动 = 启动速度快）
  - 可回收的资源包括：
    a. 缓存，如文件缓存，图片缓存
    b. 动态生成 & 添加的 View

- 典型的应用场景(两个)
![image](application_page4.png)

- 具体使用
```
registerComponentCallbacks(new ComponentCallbacks2() {
    @Override
    public void onTrimMemory(int level) {
        // Android系统会根据当前内存使用的情况，传入对应的级别
        // 下面以清除缓存为例子介绍
        super.onTrimMemory(level);
        if (level >= ComponentCallbacks2.TRIM_MEMORY_MODERATE) {
            mPendingRequests.clear();
            mBitmapHolderCache.evictAll();
            mBitmapCache.evictAll();
        }
});
```

- 可回调对象 & 对应方法
  - 特别注意：onTrimMemory() 中的 TRIM_MEMORY_UI_HIDDEN 与 onStop() 的关系
    - onTrimMemory() 中的 TRIM_MEMORY_UI_HIDDEN 的回调时刻：当应用程序中的所有 UI 组件全部不可见时
    - Activity 的 onStop() 回调时刻：当一个 Activity 完全不可见的时候
    - 使用建议：
      - 在 onStop() 中释放与 Activity 相关的资源，如取消网络连接或者注销广播接收器等
      - 在 onTrimMemory() 中的 TRIM_MEMORY_UI_HIDDEN 中释放与 UI 相关的资源，从而保证用户在使用应用程序过程中，UI 相关的资源不需要重新加载，从而提升响应速度
    - 注：onTrimMemory 的 TRIM_MEMORY_UI_HIDDEN 等级是在 onStop() 方法之前调用的
  
```
Application.onTrimMemory()
Activity.onTrimMemory()
Fragment.onTrimMemory()
Service.onTrimMemory()
ContentProvider.onTrimMemory()
```

### onLowMemory()
- 作用：监听 Android 系统整体内存较低时刻
- 调用时刻：Android 系统整体内存较低时
- 应用场景：Android 4.0 前检测内存使用情况，从而避免被系统直接杀掉 & 优化应用程序的性能体验
  - 类似于 onTrimMemory()
- 特别注意：onTrimMemory() & onLowMemory() 关系
  - onTrimMemory() 是 onLowMemory() Android 4.0 后的替代 API
  - onLowMemory() = onTrimMemory() 中的 TRIM_MEMORY_COMPLETE 级别
  - 若想兼容 Android 4.0 前，请使用 OnLowMemory(), 否则直接使用 onTrimMemory() 即可

### onConfigurationChanged()
- 作用：监听 应用程序 配置信息的改变，如屏幕旋转等
- 调用时刻：应用程序配置信息 改变时调用

```
registerComponentCallbacks(new ComponentCallbacks2() {
    @Override
    public void onConfigurationChanged(Configuration newConfig) {
        ...
    }
});
```

- 该配置信息是指：Manifest.xml 文件下的 Activity 标签属性 android:configChanges 的值，如下：
```
<activity android:name=".MainActivity">
    android:configChanges="keyboardHidden|orientation|screenSize"
    // 设置该配置属性会使 Activity 在配置改变时不重启，只执行 onConfigurationChanged()
    // 上述语句表明，设置该配置属性可使 Activity 在屏幕旋转时不重启
 </activity>
```

### registerActivityLifecycleCallbacks() & unregisterActivityLifecycleCallbacks()
- 作用：注册 / 注销对 应用程序内 所有 Activity 的生命周期监听
- 调用时刻：当应用程序内 Activity 生命周期发生变化时就会调用
  - 实际上是调用 registerActivityLifecycleCallbacks() 里 ActivityLifecycleCallbacks 接口里的方法
  
```
// 实际上需要复写的是 ActivityLifecycleCallbacks 接口里的方法
registerActivityLifecycleCallbacks(new ActivityLifecycleCallbacks() {
    @Override
    public void onActivityCreated(Activity activity, Bundle savedInstanceState) {
        Log.d(TAG,"onActivityCreated: " + activity.getLocalClassName());
    }

    @Override
    public void onActivityStarted(Activity activity) {
        Log.d(TAG,"onActivityStarted: " + activity.getLocalClassName());
    }

    @Override
    public void onActivityResumed(Activity activity) {
        Log.d(TAG,"onActivityResumed: " + activity.getLocalClassName());
    }

    @Override
    public void onActivityPaused(Activity activity) {
        Log.d(TAG,"onActivityPaused: " + activity.getLocalClassName());
    }

    @Override
    public void onActivityStopped(Activity activity) {
        Log.d(TAG, "onActivityStopped: " + activity.getLocalClassName());
    }

    @Override
    public void onActivitySaveInstanceState(Activity activity, Bundle outState) {
    }

    @Override
    public void onActivityDestroyed(Activity activity) {
        Log.d(TAG,"onActivityDestroyed: " + activity.getLocalClassName());
    }
});
```

### onTerminate()
- 调用时刻：应用程序结束时调用
  - 但该方法只用于 Android 仿真机测试，在 Android 产品机是不会调用的
  
---

## 应用场景
- 从 Applicaiton 类的方法可以看出，Applicaiton 类的应用场景有：（已按优先级排序）
  - 初始化 应用程序级别 的资源，如全局对象、环境配置变量等
  - 数据共享、数据缓存，如设置全局共享变量、方法等
  - 获取应用程序当前的内存使用情况，及时释放资源，从而避免被系统杀死
  - 监听 应用程序 配置信息的改变，如屏幕旋转等
  - 监听应用程序内 所有 Activity 的生命周期
  
---

## 具体使用
- 若需要复写实现上述方法，则需要自定义 Application 类

### 步骤1：新建 Application 子类
- 即继承 Application 类

### 步骤2：配置自定义的 Application 子类
- 在 Manifest.xml 文件中 application 标签里进行配置
```
<application
    android:name=".CarsonApplication"
    // 此处自定义 Application 子类的名字 = CarsonApplication
</application>
```

### 步骤3：使用自定义的 Application 类实例
```
private CarsonApplicaiton app;

// 只需要调用 Activity.getApplication() 或 Context.getApplicationContext() 就可以获得一个 Application 对象
app = (CarsonApplication) getApplication();

// 然后再得到相应的成员变量 或方法 即可
app.exitApp();
```

---

## 补充场景
### 应用切换至后台
- 调用 Application.onTrimMemory(int level)，这个方法包含了一个等级叫 TRIM_MEMORY_UI_HIDDEN，用于判断前后台切换
```
public class MyApplication extends Application { 
    @Override 
    public void onTrimMemory(int level) { 
        super.onTrimMemory(level); 
        if (level == TRIM_MEMORY_UI_HIDDEN) { 
            isBackground = true;
            notifyBackground(); 
        }
    }
}
```

### 手机熄屏
- 用 Intent.ACTION_SCREEN_OFF 注册 BroadcastReceiver
```
public class MyApplication extends Application {
    // ...
    @Override
    public void onCreate() {
        super.onCreate();
        // ...
        IntentFilter screenOffFilter = new IntentFilter(Intent.ACTION_SCREEN_OFF);
        registerReceiver(new BroadcastReceiver() {
            @Override
            public void onReceive(Context context, Intent intent) {
                if (isBackground) {
                    isBackground = false;
                    notifyForeground();
                }
            }
        }, screenOffFilter);
    }
}
```

### 应用切换至前台
- 没有任何 flag 或者 trim level 来判断应用切换至前台，覆写 Activity.onResume() 是最好的方法。在基类 Activity 中复写它是一个选择，但无须如此
- 利用 Application.registerActivityLifeStyleCallbacks()，可以覆写每一个生命周期函数。
```
public class MyApplication extends Application {
    // ...
    @Override
    public void onCreate() {
        super.onCreate();
        registerActivityLifecycleCallbacks(new Application.ActivityLifecycleCallbacks() {
            // ...
            @Override
            public void onActivityResumed(Activity activity) {
                if (isBackground) {
                    isBackground = false;
                    notifyForeground();
                }
            }
            // ...
        });
    }
    // ...
}
```

## 总结

![image](application_page5.png)

---